In [1]:
##python codes for the XGBoost machine learning model
#This model can calssify basaltic sample's erupted environment, subaerial or submerged eruption
#Here, we extract the major and trace elements from "Parental training dataset" to train the XGBoost model



from __future__ import division, print_function, unicode_literals

# Check if the version of python is 3.5 and above
import sys
assert sys.version_info >= (3, 5)

# Check to see if sklearn is version 0.20 and above
import sklearn #
assert sklearn.__version__ >= "0.20"
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os 
from xgboost import XGBClassifier#
from sklearn.metrics import accuracy_score, r2_score, make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV#
from sklearn.model_selection import train_test_split#
from sklearn.metrics import roc_auc_score#
from sklearn.metrics import roc_curve, auc#
from sklearn.metrics import confusion_matrix#
import seaborn as sns

np.random.seed(2022) 

# Make matplotlib diagrams work better
# matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


In [ ]:
# Ignoring Unnecessary Warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
# read Excel
#Notes
#1.put the "Table-S2.Training dataset for XGBoost modeling" and this jupyter python code into a same path

#2.Please remove the first row (the head) before applying the "Table-S2.Training dataset for XGBoost modeling".

orig_data = pd.read_excel('Table-S2.Training dataset for XGBoost modeling.xlsx')#

In [ ]:
# Separate features and labels
X = orig_data.drop(["TRUE_VALUE"], axis=1).copy()
y = orig_data["TRUE_VALUE"]

In [ ]:
plt.figure(figsize=(16, 6))
sns.countplot(orig_data.TRUE_VALUE, palette="Set2")
plt.xticks(rotation=0)

In [ ]:
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_train)

In [ ]:
#before running our model, check the imbalance problem of our trining dataset at the first
from sklearn.model_selection import cross_val_score#

# Ten-fold cross validation #
scores = cross_val_score(xgb_clf, X, y,
                        scoring = "accuracy", cv=10,#cv
                        n_jobs=-1)
def display_scores(scores):

    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(scores)

In [ ]:
#Randomly seperate the traning dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
xgb_clf.feature_importances_#

In [ ]:
# show feature importance
for feature_name, score in zip(list(X.columns), xgb_clf.feature_importances_):
    print(feature_name, ":", score)

In [ ]:
learning_rate = [ 0.1, 0.2, 0.5,0.6,0.7]
depth = [3, 4, 5, 6, 7]
min_split = [0.1,0.2,0.3,1]#
alpha1 = [0.1,0.3,0.5,0.7,0.9, 1]

In [ ]:
xgb = XGBClassifier(objective='binary:logistic',
                    eval_metric = 'auc', tree_method='hist', seed=2022,importance_type = 'cover')#

In [ ]:
xgb_cv = GridSearchCV(xgb, param_grid = {'eta': learning_rate, 'gamma': min_split, 'max_depth': depth, 'alpha':alpha1}, 
                      cv=10, scoring='f1') #cv=10: tenfold cross-validation
xgb_cv.fit(X_train, y_train)

In [ ]:
#calculate a mean value and standard deviation of the tenfold cross-validation scores of a best trained model
from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgb_cv.best_estimator_, X_train, y_train,
                        scoring = "f1", cv=10,
                        n_jobs=-1)
def display_scores(scores):
    
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(scores)

In [2]:
#show the best score (mean value)
xgb_cv.best_score_

NameError: name 'xgb_cv' is not defined

In [ ]:
#show parameters of the best model
xgb_cv.best_estimator_

In [ ]:
features = list(X.columns)
importances = xgb_cv.best_estimator_.feature_importances_
indices = np.argsort(importances)

In [ ]:
df4 = pd.DataFrame({'features':features,'importances':importances})
# df4.to_excel('ML_model_Feature_importance.xlsx')

In [ ]:
from matplotlib.pyplot import MultipleLocator
plt.barh(range(len(indices)), importances[indices], color='c', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices], fontsize=12)
plt.xticks(fontsize=20)
plt.xlabel('Relative Importance',fontsize=25)
x_major_locator=MultipleLocator(0.02)#
ax = plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim((0,0.12))
# plt.savefig('ML_model_Feature_importance.pdf', dpi=500)
plt.show()
plt.rcParams["figure.figsize"] = (20, 10)

In [ ]:
# predict the test data set
xgb_test = xgb_cv.best_estimator_
xgb_test.fit(X_train,y_train)
y_test_pred = xgb_test.predict(X_test)

In [ ]:
print('Accuracy: %.4f' % accuracy_score(y_test, y_test_pred))
print('ROC AUC: %.4f' % roc_auc_score(y_test, y_test_pred))
print('Precision: %.4f' % precision_score(y_true=y_test, y_pred=y_test_pred))
print('Recall: %.4f' % recall_score(y_true=y_test, y_pred=y_test_pred))
print('F1 Score: %.4f' % f1_score(y_true=y_test, y_pred=y_test_pred))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred=y_test_pred))

In [ ]:
confmat = confusion_matrix(y_true=y_test, y_pred=y_test_pred)

print(confmat)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i,j], va='center', ha='center')
plt.xlabel('predicted label',fontsize=20)
plt.ylabel('true label',fontsize=20)
# plt.savefig('ML_model_confusion_matrix.pdf', dpi=800)
plt.show()

In [ ]:
#predict the application dataset that is non label data

#We extract "Table-S4.Application dataset for prediction" from parental application dataset

#Please remove the first row (the head) before applying the "Table-S4.Application dataset for prediction".
predict_data = pd.read_excel ('Table-S4.Application dataset for prediction.xlsx')
x_predict_data = predict_data # process.fit_transform(predict_data)

In [ ]:
predict_results = xgb_cv.best_estimator_.predict_proba(x_predict_data)

In [ ]:
predict_results

In [ ]:
df = pd.DataFrame(data=predict_results)
#df.to_excel('predict_results.xlsx')

In [ ]:
###Xgboost end